In [1]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5519.7/8062.4 GB disk)


In [3]:
from ultralytics import YOLO
from IPython.display import display, Image

# Import datasets from roboflow

In [5]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [6]:
%cd {HOME}/datasets
!pip install roboflow
from roboflow import Roboflow

rf = Roboflow(api_key="A88aglIXim3OCSF5vNOe")
project = rf.workspace("gabriel-advent-hobto").project("detect-tdz36")
version = project.version(1)
dataset = version.download("yolov8")


/kaggle/working/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing installatio


Extracting Dataset Version Zip to detect-2 in yolov8:: 100%|██████████| 6822/6822 [00:00<00:00, 8372.01it/s]


# Train Grid Search

In [5]:
import subprocess
from itertools import product

param_grid = {
    'model': ['yolov8s.pt'],
    'dropout': [0.2, 0.5],
    'batch': [32, 64],
    'lr': [0.001, 0.0001],
    'optimizer' : ['Adam', 'RMSProp'],
}

# Generate semua kombinasi
param_combinations = list(product(*param_grid.values()))
print(len(param_combinations))

32


In [ ]:
# Memasukan semua kombinasi parameter ke dalam commands

commands = []
for params in param_combinations:
    model, dropout, batch, lr, optimizer = params
    command = f"yolo detect train \
data='/kaggle/working/datasets/detect-2/data.yaml' \
model='{model}' \
epochs=200 \
imgsz='416' \
optimizer={optimizer} \
lr0={lr} \
batch='{batch}' \
dropout='{dropout}'\
weight_decay=0.0005 \
momentum=0.937 \
plots=True \
save=True"
    commands.append(command)

In [ ]:
%cd {HOME}

import subprocess

# Loop setiap perintah dan jalankan menggunakan subprocess
for command in commands:
    print(command)
    subprocess.run(command, shell=True)

# Validasi

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model='/kaggle/working/runs/detect/train/weights/best.pt' data='/kaggle/working/datasets/detect-2/data.yaml' plots=True save_json=True

# Testing

In [ ]:
from ultralytics.yolo.engine.model import YOLO
import pandas as pd
import glob
from IPython.display import Image, display

In [ ]:
test_path = "/kaggle/working/detect-2/test/images"

model = YOLO(f"/kaggle/input/objeckdetect/pytorch/object-detect/1/best.pt")
results = model.predict(source=test_path)
names = model.names
labels = []

# DataFrame untuk menyimpan hasil deteksi
df_results = pd.DataFrame(columns=['Image', 'Labels'])

for r in results:
    # Get bounding boxes and class indices
    boxes = r.boxes.xyxy
    cls = r.boxes.cls
    
    # Sort bounding boxes and class indices by x1 coordinate
    sorted_indices = boxes[:, 0].argsort()
    sorted_boxes = boxes[sorted_indices]
    sorted_cls = cls[sorted_indices]
    
    # Get class names
    file_labels = [names[int(c)] for c in sorted_cls]
    
    # Append labels to list
    labels.append(file_labels)

In [ ]:
i = 0

# List of supported image extensions
image_extensions = ['png', 'jpg', 'jpeg']

for image_extension in image_extensions:
    image_paths = glob.glob(f"{test_path}/*.{image_extension}")
    for image_path in sorted(image_paths):
        print(image_path)
        print(labels[i])
        display(Image(filename=image_path, width=200))
        print("\n")
        
        # Tambahkan hasil deteksi ke DataFrame
        df_results = pd.concat([df_results, pd.DataFrame({'Image': [image_path], 'Labels': [', '.join(labels[i])]})], ignore_index=True)
        
        i += 1

# Simpan DataFrame ke file Excel
df_results.to_excel("/kaggle/working/detection_results.xlsx", index=False)